<h1>Example post quantum secure communication between ALice and Bob</h1>

1. Required prior knowledge of the buffer size used, for all the parties.

In [33]:
buffer_size = 3200 # y of (y X y)
sha_key_str = 'Tuesday Evening'

2. QuantCrypt provides Kyber1024 key encapsulation mechanism.

In [34]:
from quantcrypt.kem import Kyber

kem = Kyber()

3. Alice generates key pair.

In [35]:
public_key, secret_key = kem.keygen()

4. Bob generates cipher and shared secret using Alice's public key.

In [36]:
cipher_text, shared_secret = kem.encaps(public_key)


5. Alice decrypts cipher she receives from Bob to extract shared secret.

In [37]:

shared_secret_copy = kem.decaps(secret_key, cipher_text)


6. Bob encrypts the message to be sent using the shared_secret.

In [38]:
from selective_encryption import encrypt
from utils import time_it

encrypt(file_path='data/files/landscape.png',
           sha_key_str=sha_key_str, 
           buffer=buffer_size,
           aes_key=shared_secret)

itr_count: 0
dwt2d: 0.008236500
dwt2d: 0.002578959
sha256_with_key: 0.000331292
sha512_with_key: 0.001536875
xor_with_sha_key: 0.002297417
xor_with_sha_key: 0.008470625
aes_encrypt: 0.002767084
_se_encrypt: 0.032414292
itr_count: 1
dwt2d: 0.006771166
dwt2d: 0.001922167
sha256_with_key: 0.000839958
sha512_with_key: 0.001243875
xor_with_sha_key: 0.001805500
xor_with_sha_key: 0.007293166
aes_encrypt: 0.002345458
_se_encrypt: 0.027165333
encrypt: 0.066610833


7. Alice decrypts the message using shared_secret_copy.

In [39]:
from selective_encryption import decrypt

decrypt(file_PF= 'data/files/enc/landscape.png_PF.npy',
           file_PPF_1= 'data/files/enc/landscape.png_PPF_1.npy',
           file_PPF_2= 'data/files/enc/landscape.png_PPF_2.npy',
           buffer=buffer_size,
           output_file_path='data/files/dec/landscape.png',
           sha_key_str=sha_key_str, 
           aes_key=shared_secret_copy)

aes_decrypt: 0.002476334
sha256_with_key: 0.000227459
xor_with_sha_key: 0.001655083
sha512_with_key: 0.001123708
xor_with_sha_key: 0.007348875
idwt2d: 0.003026541
idwt2d: 0.011951291
aes_decrypt: 0.002383125
sha256_with_key: 0.000227750
xor_with_sha_key: 0.001895458
sha512_with_key: 0.001132875
xor_with_sha_key: 0.006885375
idwt2d: 0.003161833
idwt2d: 0.011031500
decrypt: 2.459122666


In [40]:
import os
import base64
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from utils import time_it
from selective_encryption import _se_encrypt
import oqs

KEY_SIZE = 32
BLOCK_SIZE = AES.block_size
BUFFER_SIZE = 3200

@time_it
def encrypt_message(message, key, iv):
    cipher = AES.new(key, AES.MODE_CBC, iv)
    
    padded_message = message + (BLOCK_SIZE - len(message) % BLOCK_SIZE) * b' '
    
    ciphertext = cipher.encrypt(padded_message)
    
    return base64.b64encode(iv + ciphertext)


def encrypt_file(input_file, output_file, key):
    iv = get_random_bytes(BLOCK_SIZE)
    
    with open(input_file, 'rb') as f_in:
        chunk = f_in.read(BUFFER_SIZE*BUFFER_SIZE)
        
        if len(chunk) < BUFFER_SIZE:
            chunk = f_in.read()
    
    %timeit -n 10 -r 5 encrypt_message(chunk, key, iv)
    
    %timeit -n 10 -r 5 _se_encrypt(chunk=chunk, buffer=BUFFER_SIZE, sha_key='Tuesday Evening'.encode(), ll2_enc_key=key)


input_file = 'images/64MB_image.png'
output_file = 'images/enc/4MB_image.png.enc'

kem = "ML-KEM-1024"
with oqs.KeyEncapsulation(kem) as receiver:
    with oqs.KeyEncapsulation(kem) as sender:
        public_key_receiver = receiver.generate_keypair()
        cipher, secret = sender.encap_secret(public_key_receiver)
        encrypt_file(input_file, output_file, secret)
        print(f"Encrypted {input_file} -> {output_file}")


encrypt_message: 0.045697750
encrypt_message: 0.047036750
encrypt_message: 0.046789375
encrypt_message: 0.046623375
encrypt_message: 0.046094334
encrypt_message: 0.044767542
encrypt_message: 0.047313125
encrypt_message: 0.045076458
encrypt_message: 0.045222834
encrypt_message: 0.045120875
encrypt_message: 0.044882375
encrypt_message: 0.044883125
encrypt_message: 0.044601583
encrypt_message: 0.045233542
encrypt_message: 0.045577792
encrypt_message: 0.045040875
encrypt_message: 0.044204500
encrypt_message: 0.044385917
encrypt_message: 0.044102208
encrypt_message: 0.044621625
encrypt_message: 0.044727208
encrypt_message: 0.043858625
encrypt_message: 0.044716250
encrypt_message: 0.044478250
encrypt_message: 0.045207792
encrypt_message: 0.044350917
encrypt_message: 0.045866333
encrypt_message: 0.046594250
encrypt_message: 0.044553416
encrypt_message: 0.045133750
encrypt_message: 0.047503291
encrypt_message: 0.046958833
encrypt_message: 0.046563375
encrypt_message: 0.046623792
encrypt_messag